In [50]:
import numpy as np
import pandas as pd

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    smooth_path = "/home/filtheo/smooth/"
    
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

load_smooth_dev()


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
ℹ Loading smooth


R[write to console]: This is package "smooth", v4.3.2.41001




Smooth package loaded in development mode


# Two-Stage Initialization Tests - R Implementation

Two-stage initialization works by:
1. First running a model with `initial="two-stage"` (full backcasting) to get good starting values
2. Using those values as initial guesses for optimization, allowing parameter refinement

This should produce results that are:
- Different from pure backcasting (since parameters are refined)
- Different from pure optimal (since starting values are better)
- Generally better or similar quality to both methods


### Test 1: Global level ETS(A,N,N) - two-stage vs optimal vs backcasting


In [51]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [52]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'optimal')
cat('ETS(A,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with optimal initial:
Parameters: 0.02502502 98.50918 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134
[10] 101.134 101.134 101.134


In [53]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'backcasting', nIterations = 2)
cat('ETS(A,N,N) with backcasting (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with backcasting (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [54]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'complete')
cat('ETS(A,N,N) with complete (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with complete (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 


      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [55]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with two-stage initialization (two-stage):
Parameters: 0 99.61911 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911


### Test 2: Local trend ETS(A,A,N)


In [56]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [57]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'optimal')
cat('ETS(A,A,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,N) with optimal initial:
Parameters: 0.1971955 0.004651387 100.6095 0.2576178 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.4791 166.1722 166.8654 167.5585 168.2516 168.9448 169.6379 170.3310
 [9] 171.0242 171.7173 172.4104 173.1035

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [58]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'complete')
cat('ETS(A,A,N) with complete initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] 


ETS(A,A,N) with complete initial:
Parameters: 0.2007559 0 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2649 165.8912 166.5174 167.1437 167.7699 168.3961 169.0224 169.6486
 [9] 170.2749 170.9011 171.5273 172.1536

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [59]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,N) with two-stage initialization (two-stage):
Parameters: 0.4527053 0 60.91052 -0.1539363 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.0947 164.9407 164.7868 164.6329 164.4789 164.3250 164.1711 164.0171
 [9] 163.8632 163.7092 163.5553 163.4014

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 3: Seasonal data ETS(A,A,A)


In [60]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [61]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(A,A,A) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with optimal initial:
Alpha, Beta, Gamma: 0.07547122 0.01237746 0.0009072046 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 141.9894 154.8596 170.4143 202.7312 126.5584 164.2922 156.1874 150.7438
 [9] 241.5742 180.8632 138.8821 208.7774

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [62]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(A,A,A) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with backcasting initial:
Alpha, Beta, Gamma: 0.1046524 0 0.07079328 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.8210 157.4031 172.9867 205.9609 125.8171 163.5803 154.9899 148.4708
 [9] 239.7119 180.6099 134.2689 203.4383

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [63]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(A,A,A) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with complete initial:
Alpha, Beta, Gamma: 0.1046524 0 0.07079328 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.8210 157.4031 172.9867 205.9609 125.8171 163.5803 154.9899 148.4708
 [9] 239.7119 180.6099 134.2689 203.4383

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [64]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,A) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.1870092 0 0.000812271 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 150.3658 164.7257 177.2400 210.8623 133.7236 170.5628 161.7585 155.1007
 [9] 246.3371 186.4022 142.3022 212.8921

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 4: Damped trend ETS(A,Ad,N)


In [65]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'optimal')
cat('ETS(A,Ad,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with optimal initial:
Parameters: 0.0002345491 9.622769e-06 0.9786888 200.8303 -0.7423182 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 169.2050 169.1530 169.1022 169.0524 169.0037 168.9560 168.9093 168.8636
 [9] 168.8189 168.7752 168.7323 168.6904

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [66]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'backcasting')
cat('ETS(A,Ad,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with backcasting initial:
Parameters: 0.002794672 0 1 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 167.3715 167.2025 167.0334 166.8644 166.6954 166.5264 166.3574 166.1883
 [9] 166.0193 165.8503 165.6813 165.5123

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [67]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,Ad,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with two-stage initialization (two-stage):
Parameters: 0 0 0.2559636 179.0505 -0.5540958 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 178.8599 178.8599 178.8599 178.8599 178.8599 178.8599 178.8599 178.8599
 [9] 178.8599 178.8599 178.8599 178.8599

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 5: Multiplicative error ETS(M,N,N)


In [68]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with optimal initial:
Parameters: 0.03293295 189.757 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432
 [9] 174.2432 174.2432 174.2432 174.2432

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [69]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,N,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with backcasting initial:
Parameters: 0.03312988 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972
 [9] 174.1972 174.1972 174.1972 174.1972

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [70]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with two-stage initialization (two-stage):
Parameters: 0.03303208 1065.217 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.235 174.235 174.235 174.235 174.235 174.235 174.235 174.235 174.235
[10] 174.235 174.235 174.235

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 6: Multiplicative seasonal ETS(M,A,M)


In [71]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,A,M) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with optimal initial:
Alpha, Beta, Gamma: 0.1160812 0.004623054 0.000360059 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.1198 159.0073 174.1266 202.4919 129.1491 165.7924 155.5176 152.5447
 [9] 237.5756 178.9442 137.9211 207.1010

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [72]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,A,M) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with backcasting initial:
Alpha, Beta, Gamma: 0.09873239 0 0.06260129 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.5750 158.3557 172.9124 203.5333 128.4102 163.9190 155.7270 149.5907
 [9] 236.1275 179.8825 136.0165 200.8703

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [73]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(M,A,M) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with complete initial:
Alpha, Beta, Gamma: 0.09873239 0 0.06260129 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.5750 158.3557 172.9124 203.5333 128.4102 163.9190 155.7270 149.5907
 [9] 236.1275 179.8825 136.0165 200.8703

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [74]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,A,M) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.1857728 0 0.0009670556 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 149.5835 165.0071 181.9989 211.4155 133.6198 173.1179 161.7513 157.2678
 [9] 246.5222 187.1359 144.1107 215.4362

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



## Summary

Two-stage initialization successfully:
- Runs a two-stage backcasting model first
- Extracts parameters (persistence, damping, ARMA)
- Extracts and normalizes initial states
- Uses these as starting values for optimization
- Produces results that combine benefits of both approaches
